<a href="https://colab.research.google.com/github/nitinvedwal/Bird-Classification/blob/main/birdclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
len(os.listdir("/content/drive/My Drive/bird_f/train"))

6009

In [ ]:
import pandas as pd
df_train_data = pd.read_csv("/content/drive/MyDrive/bird_f/train_index.csv")
print(len(df_train_data))

6009


In [ ]:
len(os.listdir("/content/drive/My Drive/bird_f/test"))

1480

In [ ]:
"""Importing nc=ecessery libraries"""
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
import random
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
"""Function for building th model"""
def build_model(size, num_classes):
    inputs = Input((size, size, 3))
    backbone = MobileNetV2(input_tensor=inputs, include_top=False, weights="imagenet") #Used mobilenetV2 for better performance of our model.
    backbone.trainable = True
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation="relu")(x)
    
    #x = Dense(1024, activation="relu")(x)
    #x = Dense(1024, activation="relu")(x) # No need to dence the layers because we are using MobileNetV2
    x = Dense(num_classes, activation="softmax")(x)


    model = tf.keras.Model(inputs, x)
    return model
    """Function for reading images"""
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def parse_data(x, y):
    x = x.decode()

    num_class = 211
    size = 224

    image = read_image(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)

    return image, label

def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((211))
    return x, y

"""Function for taking the image locations and taking out informations and storing them"""
def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset
path = "/content/drive/My Drive/bird_f" # Path of directory containing train,test and labels
train_path = os.path.join(path, "train/*") #path of train dataset
#test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "train_index.csv") #labels of train datasets


labels_df = pd.read_csv(labels_path) #Reading the train label
name = labels_df["name"].unique() # Making list of unique elements of breed of birds
print("Number of name: ", len(name))

name2index = {name: i for i, name in enumerate(name)} #making dictionary for birds name with index
index2name = {i: name for i, name in enumerate(name)} # vice-versa of breed2id
 

ids = glob(train_path) # randomly shuffling and giving the path of train_path
labels = []

#Reading every image and taking their breed
for image_id in ids: 
    image_id = image_id.split("/")[-1].split(".")[0]
    # breed_name = list(labels_df[(labels_df.index == image_id)]["name"])[0]
    if(len(image_id.split("_"))==4):
      breed_name = image_id.split("_")[0] + "_" + image_id.split("_")[1] + "_" + image_id.split("_")[2]
    if(len(image_id.split("_"))==3):
      breed_name = image_id.split("_")[0] + "_" + image_id.split("_")[1]
    if(len(image_id.split("_"))==2):
      breed_name = image_id.split("_")[0]
    breed_idx = name2index[breed_name]
    labels.append(breed_idx)
    """Splitting the data for training and validation"""
train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)


size = 224
num_classes = 211 # no. of breeds of birds
lr = 1e-5 # learning rate of our model
batch = 16
epochs = 20

    ## Building the DL model from the function created
model = build_model(size, num_classes)
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
    

    ## Taking the useful data from images
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

    ## Training
callbacks = [
        ModelCheckpoint("model3.h5", verbose=1, save_best_only=True), #saving the model for further use 
        ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-6)
        ]
train_steps = (len(train_x)//batch) + 1
valid_steps = (len(valid_x)//batch) + 1
#Final training the model
model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=callbacks)

Number of name:  211
9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/20
301/301 [==============================] - ETA: 0s - loss: 3.7495 - acc: 0.2565
Epoch 1: val_loss improved from inf to 3.88145, saving model to model3.h5
301/301 [==============================] - 241s 725ms/step - loss: 3.7495 - acc: 0.2565 - val_loss: 3.8814 - val_acc: 0.1248 - lr: 1.0000e-05
Epoch 2/20
301/301 [==============================] - ETA: 0s - loss: 2.6385 - acc: 0.3728
Epoch 2: val_loss improved from 3.88145 to 3.53392, saving model to model3.h5
301/301 [==============================] - 88s 294ms/step - loss: 2.6385 - acc: 0.3728 - val_loss: 3.5339 - val_acc: 0.1839 - lr: 1.0000e-05
Epoch 3/20
301/301 [==============================] - ETA: 0s - loss: 2.2450 - acc: 0.4583
Epoch 3: val_loss improved from 3.53392 to 3.26472, saving model to model3.h5
301/301 [==============================] - 85s 283ms/step - loss: 2.2450 - acc: 0.4583 - val_loss: 3.2647 - val_acc: 0.2729 - lr: 

In [ ]:
"""Testing of the data and calculating the accuracy for testing data"""
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 


"""Function for reading the images"""
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size)) # Resizing the images for equality 
    image = image / 255.0
    image = image.astype(np.float32)
    return image

path = "/content/drive/MyDrive/bird_f"
#train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*") #path of test dataset
labels_path2 = os.path.join(path, "test_index.csv") #path of test_labels


labels_df2 = pd.read_csv(labels_path2)
#breed = labels_df["breed"].unique()
#print("Number of Breed: ", len(breed))

#breed2id = {name: i for i, name in enumerate(breed)}
#id2breed = {i: name for i, name in enumerate(breed)}


"""Calling our model"""
model = tf.keras.models.load_model("model3.h5")

#for i, path in tqdm(enumerate(valid_x[:10])):
l1 = labels_df2["index"] # image ids
l2 = labels_df2["name"] # corresponding breed of bird

zip1 = list(zip(l1, l2))

random.shuffle((zip1)) # shuffling our data for better results

res1 = list(zip(*zip1))

l1 = list(res1[0])
l2 = list(res1[1])

predicted=[] # predicted array for breed of images
for i in range(len(l1)):
    image = read_image(path + "/" + "test/" + str(l1[i]) + ".png", 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_idx = np.argmax(pred)
    breed_name = index2name[label_idx]
    predicted.append(breed_name)
    
print(accuracy_score(l2,predicted)) # Calculating the accuracy for testing data

1/1 [==============================] - 0s 23ms/step
0.8702702702702703
